Это домашнее задание проходит в формате peer-review. Это означает, что его будут проверять ваши однокурсники. Поэтому пишите разборчивый код, добавляйте комментарии и пишите выводы после проделанной работы.

В этом задании вы будете решать задачу классификации математических задач по темам (многоклассовая классификация) с помощью Transformer.

В качестве датасета возьмем датасет математических задач по разным темам. Нам необходим следующий файл:

[Файл с классами](https://docs.google.com/spreadsheets/d/1IMRxByfg7gjoZ5i7rxvuNDvSrbdOJOc-/edit?usp=drive_link&ouid=104379615679964018037&rtpof=true&sd=true)

**Hint:** не перезаписывайте модели, которые вы получите на каждом из этапов этого дз. Они ещё понадобятся.

## NEW

In [2]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets
from typing import List, Union
import copy
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [20 lines of output]
  [WARNING] Unable to import torch, pre-compiling ops will be disabled. Please visit https://pytorch.org/ to see how to properly install torch on your system.
   [WARNING]  unable to import torch, please install it if you want to pre-compile any deepspeed ops.
  DS_BUILD_OPS=1
  Traceback (most recent call last):
    File "c:\users\zenfo\appdata\local\programs\python\python38\lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "c:\users\zenfo\appdata\local\programs\python\python38\lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
    File "c:\users\zenfo\appdata\local\programs\python\python38\lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118,

  Using cached accelerate-0.29.2-py3-none-any.whl.metadata (18 kB)
  Using cached deepspeed-0.14.0.tar.gz (1.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


In [16]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="/kaggle/input/datamath/data_problems.xlsx - Sheet1.csv")

dataset = dataset.rename_column("Задача", "task").rename_column("Тема", "theme")

train_valid_split = dataset['train'].train_test_split(test_size=0.2, seed=42)

from datasets import DatasetDict
dataset_tv = DatasetDict({'train': train_valid_split['train'],
                            'valid': train_valid_split['test']})

labels = ['Графы', 'Геометрия', 'Многочлен', 'Комбинаторика', 'Теория чисел',
       'Инвариант', 'Дирихле']

str_to_int = {label: i for i, label in enumerate(labels)}

FileNotFoundError: Unable to find '/kaggle/input/datamath/data_problems.xlsx - Sheet1.csv'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [3]:
class TransformerClassificationModel(nn.Module):
    def __init__(self, base_transformer_model: Union[str, nn.Module], p_dropout, num_labels):
        super().__init__()
        self.backbone = base_transformer_model
        # size_out_features = base_transformer_model.encoder.layer[0].output.dense.out_features
        size_backbone_out_f = self.backbone.pooler.dense.out_features
        self.fc1 = nn.Linear(size_backbone_out_f, size_backbone_out_f // 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=p_dropout)
        self.fc2 = nn.Linear(size_backbone_out_f // 2, num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.backbone(input_ids, attention_mask, token_type_ids).pooler_output
        x = self.fc1(output_1)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return log

In [15]:
class BertClassificationMathTrainer:
    def __init__(self, model_name, max_length=256,  batch_size=32, 
                      num_labels=7, p_dropout=0.1, lr=2e-4,
                      num_epochs=3, freeze_backbone=False):
        self.freeze_backbone = freeze_backbone
        self.model_name = model_name
        self.p_dropout = p_dropout
        self.max_length = max_length
        self.batch_size = batch_size
        self.lr = lr
        self.num_labels = num_labels
        self.num_epochs = num_epochs

    def setup_model_preprocess_data(self):
        model_orig = transformers.AutoModel.from_pretrained(self.model_name)
        tokenizer = transformers.AutoTokenizer.from_pretrained(self.model_name)

        transformer_model = TransformerClassificationModel(model_orig, self.p_dropout, self.num_labels)
    
        if self.freeze_backbone:
            transformer_model = freeze_backbone_function(transformer_model)

        self.model = copy.copy(transformer_model)
        self.model.to(device)

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)

        def preprocess_function(examples):
            result = tokenizer(
                examples['task'],     #токенизируемый текст1
                padding='max_length',  #padding до макс. длины
                max_length=self.max_length, #максимальная длина предложения
                truncation=True        #усечение до максимальной длине, указанной в max_length
            )
            result['label'] = [str_to_int[label] for label in examples['theme']]
            return result
        
        self.dataset_preprocessed = dataset_tv.map(preprocess_function, batched=True)
        

    def train(self):
        train_set =  self.dataset_preprocessed['train']
        train_loader = torch.utils.data.DataLoader(
            train_set, batch_size=self.batch_size, shuffle=False, collate_fn=transformers.default_data_collator
        )

        self.model.train()

        predictions, targets = [], []

        losses_train = []
        
        for epoch in range(self.num_epochs):
                losses_batch = [] 
            
                for batch in tqdm(train_loader):
                    predicted = self.model(
                        batch['input_ids'].to(device),
                        batch['attention_mask'].to(device),
                        batch['token_type_ids'].to(device)
                    )

                    outputs = torch.softmax(predicted, dim=1)
                    loss = self.criterion(outputs, batch['labels'].to(device))
                    losses_batch.extend(loss.item())
                   
                    self.loss.backward()
                    self.optimizer.step()
                    self.optimizer.zero_grad()

                    preds = torch.max(outputs, 1).indices
                    predictions.extend(preds.cpu().numpy())
                    targets.extend(batch['labels'])
                   
                print(f'Epoch {epoch+1}, Train Loss: {np.mean(losses_batch)}')
                losses_train.extend(np.mean(losses_batch))
            
        print('Result accuracy on train', accuracy_score(targets, predictions))
        print('Result f1 score on train', f1_score(targets, predictions))

        return self.model


    def eval(self):
        val_set = self.dataset_preprocessed['valid']
        val_loader = torch.utils.data.DataLoader(
            val_set, batch_size=self.batch_size, shuffle=False, collate_fn=transformers.default_data_collator
        )

        self.model.eval()

        predictions, targets = [], []

        for batch in tqdm(val_loader):
            with torch.no_grad():
                predicted = self.model(
                      batch['input_ids'].to(device),
                      batch['attention_mask'].to(device),
                      batch['token_type_ids'].to(device)
                )
                
                outputs = torch.softmax(predicted, dim=1)
                preds = torch.max(outputs, 1).indices

                predictions.extend(preds.cpu().numpy())
                targets.extend(batch['labels'])

        print('Result accuracy on validation', accuracy_score(targets, predictions))
        print('Result f1 score on validation', f1_score(targets, predictions))


In [ ]:
def train_transformer(model_name, dataset_tv,
                      freeze_backbone=True, 
                      num_labels=7, p_dropout=0.1,
                      num_epochs=15, lr=2e-4, 
                      MAX_LENGTH=256, batch_size=32 ):
    
    model_orig = transformers.AutoModel.from_pretrained(model_name)
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

    def preprocess_function(examples):
        result = tokenizer(
            examples['task'],     #токенизируемый текст1
            padding='max_length',  #padding до макс. длины
            max_length=MAX_LENGTH, #максимальная длина предложения
            truncation=True        #усечение до максимальной длине, указанной в max_length
        )
        result['label'] = [str_to_int[label] for label in examples['theme']]
        return result

    dataset_preprocessed = dataset_tv.map(preprocess_function, batched=True)

    val_set = dataset_preprocessed['valid']
    val_loader = torch.utils.data.DataLoader(
        val_set, batch_size=batch_size, shuffle=False, collate_fn=transformers.default_data_collator
    )

    train_set = dataset_preprocessed['train']
    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, shuffle=False, collate_fn=transformers.default_data_collator
    )

    transformer_model = TransformerClassificationModel(model_orig, p_dropout, num_labels)
    
    if freeze_backbone:
        transformer_model = freeze_backbone_function(transformer_model)

    model = copy.copy(transformer_model)
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    

In [ ]:
def freeze_backbone_function(model: TransformerClassificationModel, freeze=True):
    for param in model.backbone.parameters():
        param.requires_grad = not freeze
    return model

In [74]:
def train_transformer(model_name, dataset_tv, freeze_backbone=True, num_epochs=15, lr=2e-4):

    model_orig = transformers.AutoModel.from_pretrained(model_name)
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    MAX_LENGTH = 256
# описание параметров: https://huggingface.co/docs/transformers/main_classes/tokenizer
    def preprocess_function(examples):
        result = tokenizer(
            examples['task'],     #токенизируемый текст1
            # examples['text2'],     #токенизируемый текст2
            padding='max_length',  #padding до макс. длины
            max_length=MAX_LENGTH, #максимальная длина предложения
            truncation=True        #усечение до максимальной длине, указанной в max_length
        )
        result['label'] = [str_to_int[label] for label in examples['theme']]
        return result

    dataset_preprocessed = dataset_tv.map(preprocess_function, batched=True)

    val_set = dataset_preprocessed['valid']
    val_loader = torch.utils.data.DataLoader(
        val_set, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
    )

    train_set = dataset_preprocessed['train']
    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
    )
    
    transformer_model = TransformerClassificationModel(model_orig)
    
    if freeze_backbone:
        transformer_model = freeze_backbone_function(transformer_model)

    model = copy.copy(transformer_model)
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()

        running_loss, running_acc = 0, 0

        preds_on_dev = []
        trues_on_dev = []
        
        losses = []

        for batch in tqdm(train_loader):

            predicted = model(
                      batch['input_ids'].to(device),
                      batch['attention_mask'].to(device),
                      batch['token_type_ids'].to(device)
            )

            outputs = torch.softmax(predicted, dim=1)
            loss = criterion(outputs, batch['labels'].to(device))
            
            loss.backward()
            
            losses.append(loss.item())

            optimizer.step()
            optimizer.zero_grad()

            preds = torch.max(outputs, 1).indices
            trues = batch['labels']

        #     preds_on_dev = torch.argmax(pred.logits, dim=-1).data
        # trues_on_dev = batch_on_dev['labels'].data

            preds_on_dev.extend(preds.data.cpu().numpy())
            trues_on_dev.extend(trues.numpy())
            

#         loss_all.append()
        print(np.mean(losses))
        
        accuracy = accuracy_score(trues_on_dev, preds_on_dev)

        #     running_loss += loss.item()
        #     running_acc += torch.sum(preds.cpu() == batch['labels'].data)
        print('train acc', accuracy)

        # epoch_loss = running_loss / len(train_set)
        # epoch_acc = running_acc.double() / len(train_set)
        # print()
        # print('{} Loss: {:.4f} Accuracy Train: {:.4f}'.format(epoch, epoch_loss, epoch_acc))

        model.eval()

        # running_acc_val = 0
        trues_on_dev_val = []
        preds_on_dev_val = []

        for batch in tqdm(val_loader):
            with torch.no_grad():
                predicted = model(
                      batch['input_ids'].to(device),
                      batch['attention_mask'].to(device),
                      batch['token_type_ids'].to(device)
                )

            outputs = torch.softmax(predicted, dim=1)
            preds = torch.max(outputs, 1).indices

                # running_acc_val += torch.sum(preds.cpu() == batch['labels'].data)

                # preds = torch.max(outputs, 1).indices
            trues = batch['labels']

        #     preds_on_dev = torch.argmax(pred.logits, dim=-1).data
        # trues_on_dev = batch_on_dev['labels'].data

            preds_on_dev_val.extend(preds.data.cpu().numpy())
            trues_on_dev_val.extend(trues.numpy())


        accuracy_val = accuracy_score(trues_on_dev_val, preds_on_dev_val)

        print('val acc', accuracy_val)

        # epoch_acc_val = running_acc_val.double() / len(val_set)
        # print()
        # print('Accuracy Valid: {:.4f}'.format(epoch, epoch_acc_val))

    return model

In [72]:
model_name = 'cointegrated/rubert-tiny2'

In [70]:
model_name = 'tbs17/MathBert'

In [75]:
model_ft_rub = train_transformer(model_name, dataset_tv)

Map:   0%|          | 0/4218 [00:00<?, ? examples/s]

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

100%|██████████| 132/132 [00:07<00:00, 18.52it/s]


1.9388891873937664
train acc 0.39734471313418684


100%|██████████| 33/33 [00:01<00:00, 20.77it/s]


...

val acc 0.43412322274881515


100%|██████████| 132/132 [00:07<00:00, 18.84it/s]


1.7355496775020252
train acc 0.4594594594594595


100%|██████████| 33/33 [00:01<00:00, 21.01it/s]

val acc 0.43412322274881515


## PREV

In [ ]:
# model_name = "gchhablani/bert-base-cased-finetuned-qqp"
model_name = 's-nlp/russian_toxicity_classifier'


In [ ]:
ёф


Map:   0%|          | 0/4218 [00:00<?, ? examples/s]

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModelForPreTraining.from_pretrained("cointegrated/rubert-tiny2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 

The secret `HF_TOKEN` does not exist in your Colab secrets.

To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.

You will be able to reuse this secret in all of your notebooks.

Please note that authentication is recommended but still optional to access public models or datasets.

  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

In [ ]:
nn.list(model.children())

In [14]:
# nn.Sequential(*list(model.children()))

In [ ]:
ctrl: CTRLModel (Salesforce CTRL model)

flaubert: FlaubertModel (Flaubert model)

electra: ElectraModel

In [15]:
# model

In [18]:
model = transformers.AutoModel.from_pretrained("CTRLModel")

OSError: CTRLModel is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [19]:
model = transformers.AutoModel.from_pretrained("bert-base-uncased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [20]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [21]:
model.pooler.dense.out_features

768

In [17]:
model = transformers.AutoModel.from_pretrained("ElectraModel")

OSError: ElectraModel is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
dataset_preprocessed

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'task', 'theme', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 4218
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'task', 'theme', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 1055
    })
})

In [ ]:
val_set = dataset_preprocessed['valid']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
)

NameError: name 'dataset_preprocessed' is not defined

In [ ]:
train_set = dataset_preprocessed['train']
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
)

In [ ]:
len(train_set)

4218

### Задание 1 (2 балла)

Напишите кастомный класс для модели трансформера для задачи классификации, использующей в качествке backbone какую-то из моделей huggingface.

Т.е. конструктор класса должен принимать на вход название модели и подгружать её из huggingface, а затем использовать в качестве backbone (достаточно возможности использовать в качестве backbone те модели, которые упомянуты в последующих пунктах)

In [44]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
device

device(type='cuda')

In [90]:
class TransformerClassificationModel(nn.Module):
    def __init__(self, base_transformer_model: Union[str, nn.Module]):
        super().__init__()
        self.backbone = nn.Sequential(*list(base_transformer_model.children())[:-1])
        size_out_features = base_transformer_model.pooler.dense.out_features
        self.fc1 = nn.Linear(size_out_features, size_out_features // 2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(size_out_features // 2, 7)

    # def forward(self, x, input_ids, attention_mask, token_type_ids):
    #     out_backbone = self.backbone(input_ids, attention_mask, token_type_ids)
    def forward(self, x):
        out_backbone = self.backbone(x)
        # outputs = self.fc2(self.relu(self.fc1(out_backbone)))
        return out_backbone

In [80]:
model = transformers.AutoModel.from_pretrained("bert-base-uncased")

In [91]:
model2 = TransformerClassificationModel(model)

In [38]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# transformer_model = TransformerClassificationModel(base_transformer_model=model_name)
dataset_preprocessed = dataset_tv.map(preprocess_function, batched=True)

val_set = dataset_preprocessed['valid']
val_loader = torch.utils.data.DataLoader(
        val_set, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
    )

train_set = dataset_preprocessed['train']
train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
    )

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/4218 [00:00<?, ? examples/s]

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

In [39]:
for batch in val_loader:
    break

In [94]:
# model2.to(device)

In [95]:
model2(
        batch['input_ids'].to(device)
                  # ,batch['attention_mask'].to(device)
                  #     ,batch['token_type_ids'].to(device)
                )

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-1.9092e-01,  5.3479e-01,  4.3190e-01,  ..., -2.7613e-01,
           4.5686e-01, -1.8627e-01],
         [-1.9024e-01, -4.7285e-01,  5.1319e-01,  ..., -1.9642e-01,
           9.8702e-01,  5.5750e-01],
         [-1.4818e-01,  2.3065e-01,  4.0601e-01,  ...,  9.7029e-02,
           1.0013e+00, -8.5465e-01],
         ...,
         [-8.7037e-04,  2.1890e-01,  6.2641e-01,  ..., -6.8165e-01,
           2.6300e-01, -1.9774e-01],
         [-3.5992e-02,  2.2777e-01,  6.1364e-01,  ..., -6.8368e-01,
           2.0283e-01, -2.4848e-01],
         [ 1.7301e-02,  2.9918e-01,  5.8846e-01,  ..., -6.0201e-01,
           2.5574e-01, -3.6229e-01]],

        [[-3.4578e-01,  4.9410e-01,  4.6351e-01,  ..., -4.4134e-01,
           3.6828e-01, -4.3976e-01],
         [-3.7312e-01,  3.3804e-01,  1.7126e-01,  ..., -3.9942e-02,
           1.5942e+00,  3.5076e-01],
         [-5.8107e-01, -5.1624e-01,  5.6596e-01,  ..., -2.8919e-01,
           7.268

In [96]:
params = list(model2.named_parameters())

In [98]:
for p in params:
  print(p[0])

backbone.0.word_embeddings.weight

backbone.0.position_embeddings.weight

backbone.0.token_type_embeddings.weight

backbone.0.LayerNorm.weight

backbone.0.LayerNorm.bias

backbone.1.layer.0.attention.self.query.weight

backbone.1.layer.0.attention.self.query.bias

backbone.1.layer.0.attention.self.key.weight

backbone.1.layer.0.attention.self.key.bias

backbone.1.layer.0.attention.self.value.weight

backbone.1.layer.0.attention.self.value.bias

backbone.1.layer.0.attention.output.dense.weight

backbone.1.layer.0.attention.output.dense.bias

backbone.1.layer.0.attention.output.LayerNorm.weight

backbone.1.layer.0.attention.output.LayerNorm.bias

backbone.1.layer.0.intermediate.dense.weight

backbone.1.layer.0.intermediate.dense.bias

backbone.1.layer.0.output.dense.weight

backbone.1.layer.0.output.dense.bias

backbone.1.layer.0.output.LayerNorm.weight

backbone.1.layer.0.output.LayerNorm.bias

backbone.1.layer.1.attention.self.query.weight

backbone.1.layer.1.attention.self.query.bias


In [103]:
len(model2.backbone[1].layer[11].output.LayerNorm.bias)

768

In [104]:
for p in list(model.named_parameters()):
  print(p[0])

embeddings.word_embeddings.weight

embeddings.position_embeddings.weight

embeddings.token_type_embeddings.weight

embeddings.LayerNorm.weight

embeddings.LayerNorm.bias

encoder.layer.0.attention.self.query.weight

encoder.layer.0.attention.self.query.bias

encoder.layer.0.attention.self.key.weight

encoder.layer.0.attention.self.key.bias

encoder.layer.0.attention.self.value.weight

encoder.layer.0.attention.self.value.bias

encoder.layer.0.attention.output.dense.weight

encoder.layer.0.attention.output.dense.bias

encoder.layer.0.attention.output.LayerNorm.weight

encoder.layer.0.attention.output.LayerNorm.bias

encoder.layer.0.intermediate.dense.weight

encoder.layer.0.intermediate.dense.bias

encoder.layer.0.output.dense.weight

encoder.layer.0.output.dense.bias

encoder.layer.0.output.LayerNorm.weight

encoder.layer.0.output.LayerNorm.bias

encoder.layer.1.attention.self.query.weight

encoder.layer.1.attention.self.query.bias

encoder.layer.1.attention.self.key.weight

encoder.la

In [105]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
# ### This is just an interface example. You may change it if you want.

# class TransformerClassificationModel(nn.Module):
#     # def __init__(self, base_transformer_model: str):
#     def __init__(base_transformer_model: Union[str, nn.Module]):
#         self.backbone = transformers.AutoModelForSequenceClassification.from_pretrained(base_transformer_model)
#         # YOUR CODE: create additional layers for classfication
#         self.fc_res = nn.Linear(2, 7)

#     def forward(self, x):
#         # YOUR CODE: propagate inputs through the model. Return dict with logits
#         # pred = self.backbone(
#         #                   input_ids=batch['input_ids'].to(device),
#         #                   attention_mask=batch['attention_mask'].to(device),
#         #                   token_type_ids=batch['token_type_ids'].to(device)
#         #                 )
#         logits = self.backbone(x)
#         outputs = self.fc_res(logits)
#         # torch.max(outputs, 1).indices
#         # outputs = torch.softmax(predicted.logits, dim=1)
#         return outputs

### Задание 2 (1 балл)

Напишите функцию заморозки backbone у модели (если необходимо, возвращайте из функции модель)

In [ ]:
def freeze_backbone_function(model: TransformerClassificationModel):
    for param in model.parameters():
        param.requires_grad = False

### Задание 3 (2 балла)

Напишите функцию, которая будет использована для тренировки (дообучения) трансформера (TransformerClassificationModel). Функция должна поддерживать обучение с замороженным и размороженным backbone.

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining

# tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
# model = AutoModelForPreTraining.from_pretrained("cointegrated/rubert-tiny2")


In [ ]:
MAX_LENGTH = 128
# описание параметров: https://huggingface.co/docs/transformers/main_classes/tokenizer
def preprocess_function(examples):
    result = tokenizer(
        examples['task'],     #токенизируемый текст1
        # examples['text2'],     #токенизируемый текст2
        padding='max_length',  #padding до макс. длины
        max_length=MAX_LENGTH, #максимальная длина предложения
        truncation=True        #усечение до максимальной длине, указанной в max_length
    )
    result['label'] = [str_to_int[label] for label in examples['theme']]
    return result

In [ ]:
model_name = "distilbert/distilbert-base-uncased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

# transformer_model = TransformerClassificationModel(base_transformer_model=model_name)
dataset_preprocessed = dataset_tv.map(preprocess_function, batched=True)

val_set = dataset_preprocessed['valid']
val_loader = torch.utils.data.DataLoader(
        val_set, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
    )

train_set = dataset_preprocessed['train']
train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
    )

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/4218 [00:00<?, ? examples/s]

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

In [ ]:
# for param in transformer_model.parameters():
#     print(param)

In [ ]:
modelb = transformers.AutoModel.from_pretrained(model_name)

In [87]:
# list(modelb.children())

In [88]:
#  modelb.transformer.layer

In [ ]:
# Предполагается, что ваша модель уже определена и называется `model`
# Получаем первый блок трансформера (например, для простоты)
first_transformer_block = modelb.transformer.layer[0]

# Получаем слой MultiHeadSelfAttention внутри этого блока
multi_head_self_attention = first_transformer_block.attention

# Получаем слой Linear, который преобразует выходные данные
output_linear_layer = multi_head_self_attention.out_lin

# Получаем количество выходных признаков
num_output_features = output_linear_layer.out_features

print(f"Количество выходных признаков: {num_output_features}")


Количество выходных признаков: 768


In [42]:
# modelb

In [ ]:
modelb.layer.ffn.lin2.out_features

AttributeError: 'DistilBertModel' object has no attribute 'layer'

In [41]:
# nn.Sequential(*list(modelb.children())[:2])

In [40]:
# modelb.to(device)

In [ ]:
for batch in val_loader:
  break

In [ ]:
batch['token_type_ids']

KeyError: 'token_type_ids'

In [ ]:
predicted = modelb(
                      input_ids=batch['input_ids'].to(device),
                      attention_mask=batch['attention_mask'].to(device)
                      # ,token_type_ids=batch['token_type_ids'].to(device)
                )

In [ ]:
predicted

BaseModelOutput(last_hidden_state=tensor([[[-0.1179,  0.1320, -0.1292,  ..., -0.2624,  0.6541,  0.5276],
         [-0.3249, -0.2021,  0.2641,  ..., -0.0628,  0.6533,  0.5706],
         [-0.3860, -0.0450,  0.1016,  ..., -0.1839,  0.6383, -0.7752],
         ...,
         [ 0.0960,  0.0731,  0.0888,  ...,  0.0945,  0.3984,  0.1422],
         [ 0.0420,  0.0839,  0.0592,  ...,  0.0427,  0.3808,  0.2085],
         [-0.1579,  0.2862, -0.0940,  ...,  0.0706,  0.4186,  0.0529]],

        [[-0.2737,  0.0359, -0.2033,  ..., -0.3044,  0.4584,  0.5245],
         [-0.0870,  0.1080,  0.1478,  ..., -0.3168,  0.9336,  0.5073],
         [-0.3931, -0.2930, -0.0032,  ..., -0.3349,  0.8640,  0.1997],
         ...,
         [-0.0513, -0.1906, -0.1408,  ..., -0.0311,  0.3712, -0.0340],
         [-0.1284, -0.2459, -0.1230,  ..., -0.3110,  0.3811,  0.0034],
         [-0.1462, -0.1260, -0.1019,  ..., -0.0978,  0.4878, -0.0165]],

        [[-0.1231,  0.0772, -0.1819,  ..., -0.2153,  0.7008,  0.6451],
         [-

In [ ]:
modelb()

In [ ]:
modelb()

In [ ]:
for batch in val_loader:
    break

In [ ]:
transformer_model.to(device)

TransformerClassificationModel(
  (backbone): BertForPreTraining(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(83828, 312, padding_idx=0)
        (position_embeddings): Embedding(2048, 312)
        (token_type_embeddings): Embedding(2, 312)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-2): 3 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=312, out_features=312, bias=True)
                (key): Linear(in_features=312, out_features=312, bias=True)
                (value): Linear(in_features=312, out_features=312, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=312, out_featur

In [ ]:
predicted = transformer_model(
                      input_ids=batch['input_ids'].to(device),
                      attention_mask=batch['attention_mask'].to(device),
                      token_type_ids=batch['token_type_ids'].to(device)
                )

In [ ]:
import copy
from tqdm import tqdm

def train_transformer(model_name, dataset_tv, freeze_backbone=True, num_epochs=10):

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    transformer_model = TransformerClassificationModel(base_transformer_model=model_name)

    dataset_preprocessed = dataset_tv.map(preprocess_function, batched=True)

    val_set = dataset_preprocessed['valid']
    val_loader = torch.utils.data.DataLoader(
        val_set, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
    )

    train_set = dataset_preprocessed['train']
    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
    )

    model = copy.copy(transformer_model)
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for epoch in range(num_epochs):
        model.train()

        running_loss, running_acc = 0, 0

        preds_on_dev = []
        trues_on_dev = []

        for batch in tqdm(train_loader):

            predicted = model(
                      input_ids=batch['input_ids'].to(device),
                      attention_mask=batch['attention_mask'].to(device),
                      token_type_ids=batch['token_type_ids'].to(device)
            )

            outputs = torch.softmax(predicted, dim=1)

            loss = criterion(outputs, batch['labels'].to(device))
            loss.backward()
            optimizer.zero_grad()

            preds = torch.max(outputs, 1).indices
            trues = batch['labels']

        #     preds_on_dev = torch.argmax(pred.logits, dim=-1).data
        # trues_on_dev = batch_on_dev['labels'].data

            preds_on_dev.extend(preds.data.cpu().numpy())
            trues_on_dev.extend(trues.numpy())


        accuracy = accuracy_score(trues_on_dev, preds_on_dev)

        #     running_loss += loss.item()
        #     running_acc += torch.sum(preds.cpu() == batch['labels'].data)
        print('train acc', accuracy)

        # epoch_loss = running_loss / len(train_set)
        # epoch_acc = running_acc.double() / len(train_set)
        # print()
        # print('{} Loss: {:.4f} Accuracy Train: {:.4f}'.format(epoch, epoch_loss, epoch_acc))

        model.eval()

        # running_acc_val = 0
        trues_on_dev_val = []
        preds_on_dev_val = []

        for batch in tqdm(val_loader):
            with torch.no_grad():
                predicted = model(
                      input_ids=batch['input_ids'].to(device),
                      attention_mask=batch['attention_mask'].to(device),
                      token_type_ids=batch['token_type_ids'].to(device)
                )

                outputs = torch.softmax(predicted, dim=1)
                preds = torch.max(outputs, 1).indices

                # running_acc_val += torch.sum(preds.cpu() == batch['labels'].data)

                # preds = torch.max(outputs, 1).indices
                trues = batch['labels']

        #     preds_on_dev = torch.argmax(pred.logits, dim=-1).data
        # trues_on_dev = batch_on_dev['labels'].data

                preds_on_dev_val.extend(preds.data.cpu().numpy())
                trues_on_dev_val.extend(trues.numpy())


        accuracy_val = accuracy_score(trues_on_dev_val, preds_on_dev_val)

        print('val acc', accuracy_val)

        # epoch_acc_val = running_acc_val.double() / len(val_set)
        # print()
        # print('Accuracy Valid: {:.4f}'.format(epoch, epoch_acc_val))

    return model

In [ ]:
# model_name = "gchhablani/bert-base-cased-finetuned-qqp"
# model_name = 's-nlp/russian_toxicity_classifier'
model_name = 'cointegrated/rubert-tiny2'

In [ ]:
# mod = TransformerClassificationModel(base_transformer_model=model_name)

In [ ]:
test = train_transformer(model_name, dataset_tv)

100%|██████████| 132/132 [00:12<00:00, 10.77it/s]


train acc 0.06258890469416785


100%|██████████| 33/33 [00:02<00:00, 14.26it/s]


val acc 0.08530805687203792


100%|██████████| 132/132 [00:12<00:00, 10.74it/s]


...

val acc 0.08530805687203792


100%|██████████| 132/132 [00:12<00:00, 10.54it/s]


train acc 0.06567093409198672


100%|██████████| 33/33 [00:02<00:00, 13.94it/s]

val acc 0.08530805687203792


In [ ]:
model_name

'cointegrated/rubert-tiny2'

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

dataset_preprocessed = dataset_tv.map(preprocess_function, batched=True)

val_set = dataset_preprocessed['valid']
val_loader = torch.utils.data.DataLoader(
        val_set, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
    )

train_set = dataset_preprocessed['train']
train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
    )


Map:   0%|          | 0/4218 [00:00<?, ? examples/s]

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

In [ ]:
for batch in val_loader:
  break

In [ ]:
# batch

In [ ]:
mod.to(device)

TransformerClassificationModel(
  (backbone): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(83828, 312, padding_idx=0)
        (position_embeddings): Embedding(2048, 312)
        (token_type_embeddings): Embedding(2, 312)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-2): 3 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=312, out_features=312, bias=True)
                (key): Linear(in_features=312, out_features=312, bias=True)
                (value): Linear(in_features=312, out_features=312, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=312,

In [ ]:
mod.to(device)

TransformerClassificationModel(
  (backbone): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(83828, 312, padding_idx=0)
        (position_embeddings): Embedding(2048, 312)
        (token_type_embeddings): Embedding(2, 312)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-2): 3 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=312, out_features=312, bias=True)
                (key): Linear(in_features=312, out_features=312, bias=True)
                (value): Linear(in_features=312, out_features=312, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=312,

In [ ]:
predicted = mod(
                      input_ids=batch['input_ids'],
                      attention_mask=batch['attention_mask'],
                      token_type_ids=batch['token_type_ids']
                )

In [ ]:
predicted = mod(
                      input_ids=batch['input_ids'].to(device),
                      attention_mask=batch['attention_mask'].to(device),
                      token_type_ids=batch['token_type_ids'].to(device)
                )

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
predicted

tensor([[-0.0726,  0.9478,  0.9904,  0.1767, -0.4818, -2.0024,  2.0181],
        [-0.0964,  0.8798,  0.9947,  0.1010, -0.4262, -1.9425,  1.9367],
        [-0.1040,  0.8238,  0.9701,  0.0750, -0.3927, -1.8807,  1.8440],
        [-0.0582,  1.0375,  1.0246,  0.2250, -0.5377, -2.0991,  2.1620],
        [-0.0608,  1.0662,  1.0525,  0.2189, -0.5484, -2.1377,  2.2237],
        [-0.1057,  0.7684,  0.9319,  0.0669, -0.3655, -1.8133,  1.7393],
        [-0.0765,  0.9520,  1.0026,  0.1653, -0.4798, -2.0116,  2.0345],
        [-0.0960,  0.8185,  0.9472,  0.0989, -0.3984, -1.8657,  1.8162],
        [-0.0813,  0.9460,  1.0094,  0.1503, -0.4721, -2.0092,  2.0333],
        [-0.0683,  1.0466,  1.0551,  0.1953, -0.5317, -2.1210,  2.2014],
        [-0.0524,  1.0793,  1.0428,  0.2448, -0.5629, -2.1452,  2.2312],
        [-0.0751,  0.8947,  0.9559,  0.1662, -0.4548, -1.9387,  1.9198],
        [-0.0580,  1.0579,  1.0396,  0.2268, -0.5474, -2.1243,  2.2014],
        [ 0.0196,  1.4368,  1.1448,  0.4818, -0.800

### Задание 4 (1 балл)

Проверьте вашу функцию из предыдущего пункта, дообучив двумя способами
*cointegrated/rubert-tiny2* из huggingface.

In [ ]:
rubert_tiny_transformer_model = #...
rubert_tiny_finetuned_with_freezed_backbone = train_transformer(rubert_tiny_transformer_model, freeze_backbone=True)

rubert_tiny_transformer_model = #...
rubert_tiny_full_finetuned = train_transformer(rubert_tiny_transformer_model, freeze_backbone=False)

### Задание 5 (1 балл)

Обучите *tbs17/MathBert* (с замороженным backbone и без заморозки), проанализируйте результаты. Сравните скоры с первым заданием. Получилось лучше или нет? Почему?

In [ ]:
### YOUR CODE IS HERE (probably, similar on the previous step)

### Задание 6 (1 балл)

Напишите функцию для отрисовки карт внимания первого слоя для моделей из задания

In [ ]:
def draw_first_layer_attention_maps(attention_head_ids: List, text: str, model: TransformerClassificationModel):
    pass

### Задание 7 (1 балл)

Проведите инференс для всех моделей **ДО ДООБУЧЕНИЯ** на 2-3 текстах из датасета. Посмотрите на головы Attention первого слоя в каждой модели на выбранных текстах (отрисуйте их отдельно).

Попробуйте их проинтерпретировать. Какие связи улавливают карты внимания? (если в модели много голов Attention, то проинтерпретируйте наиболее интересные)

In [ ]:
### YOUR CODE IS HERE

### Задание 8 (1 балл)

Сделайте то же самое для дообученных моделей. Изменились ли карты внимания и связи, которые они улавливают? Почему?

In [ ]:
### YOUR CODE IS HERE